### 📚 __Introduction__

You work for a startup that sells food products. You need to research user behavior for the company's app.

First, study the sales funnel. Find out how users progress through the purchase process. How many users actually reach the purchase stage? How many get stuck at earlier stages? Which stages in particular?

Next, analyze the results of an A/A/B test (read on for more information about A/A/B tests). The design team would like to change the fonts throughout the app, but management fears that users might find the new design intimidating. Therefore, they decide to base their decision on the results of an A/A/B test.

Users are divided into three groups: two control groups receive the old fonts, and one test group receives the new fonts. Determine which font set produces better results.

Creating two control groups has certain advantages. We can establish the principle that we will only trust the accuracy of our tests when the two control groups are similar. If there are significant differences between the control groups, this can help us identify factors that might be skewing the results. Comparing the control groups also tells us how much time and data we will need when conducting further tests.

You will use the same dataset for general analysis and for the A/A/B analysis. In real projects, experiments are constantly being conducted. The analytics team studies app quality using general data, without paying attention to whether users are participating in experiments.

__Data Description__
Each log entry is a user action or event.

- EventName: The name of the event.
- DeviceIDHash: A unique user identifier.
- EventTimestamp: The time of the event.
- ExpId: The experiment ID. 246 and 247 are the control groups, and 248 is the test group. Instructions for completing the project

__Step 1__. _Open the data file and read the general information_

File path: */datasets/logs_exp_us.csv* Download the dataset

__Step 2__. _Prepare the data for analysis_

- Rename the columns in a way that suits you.
- Check the data types and missing values. Correct the data if necessary.
- Add a date and time column and a separate column for dates.

__Step 3__. _Study and verify the data_

- How many events are in the logs?
- How many users are in the logs?
- What is the average number of events per user?
- What time period does the data cover? Find the earliest and latest dates. Plot a histogram by date and time. Can you be sure that you have equally complete data for the entire period? Older events might end up in some users' logs for technical reasons, and this could skew the overall picture. Find the point at which the data becomes complete and ignore the earlier section. What period does the data actually represent?
- Did you lose many events and users by excluding the oldest data?
- Make sure you have users from all three experimental groups.

__Step 4__. _Study the event funnel_

- Look at what events are in the logs and their occurrence frequency. Sort them by frequency.
- Find the number of users who performed each of these actions. Sort the events by the number of users. Calculate the percentage of users who performed the action at least once.
- In what order do you think the actions occurred? Are they all part of a single sequence? You don't need to consider them when calculating the funnel.
- Use the event funnel to find the percentage of users who move from one stage to the next. For example, for the event sequence A → B → C, calculate the percentage of users in stage B relative to the number of users in stage A, and the percentage of users in stage C relative to the number in stage B.
- At which stage do you lose the most users? What percentage of users complete the entire journey from their first event to making a purchase?

__Step 5__. _Analyze the experiment results_

- How many users are in each group?
- We have two control groups in the A/A test, where we verify our mechanisms and calculations. Check if there is a statistically significant difference between samples 246 and 247.
- Select the most popular event. In each of the control groups, find the number of users who performed this action. Calculate the percentage. Check if the difference between the groups is statistically significant. Repeat this process for all other events (you'll save time if you create a special function for this test). Can you confirm that the groups were divided correctly?
- Do the same for the group with the modified data source. Compare the results with those of each control group for each event individually. Compare the results with the combined results of the control groups. What conclusions can you draw from the experiment?
- What significance level did you set for testing the statistical hypotheses mentioned earlier? Calculate how many statistical hypothesis tests you performed. With a statistical significance level of 0.1, one out of every 10 results could be false. What should the significance level be? If you want to change it, repeat the previous steps and re-evaluate your conclusions.

### 💻 __1. Notebook Libraries and Customization__

In [1]:
from IPython.display import display, HTML
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
from scipy import stats as st
from scipy.stats import gaussian_kde
from statsmodels.stats.proportion import proportions_ztest
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.max_rows', 25)  # display max 25 rows
pd.set_option('display.max_columns', 25)  # display max 10 columns
# show max 15 characters in each column
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 150)  # Display max 150 characters in cell

### 💻 __2. Functions__

In [2]:
# Function used to detect missing values ​​not treated as NAN due to the use of keep_defaukt_na=False
def missing_values_check(df):

    missing_values = ['', ' ', 'N/A', 'none', 'None',
                      'null', 'NULL', 'NaN', 'nan', 'NAN', 'nat', 'NaT']

    for column in df.columns:

        if df[column].isin(missing_values).any():

            display(HTML(
                f"> Null values ​​in column <i>'{column}'</i>: <b>{df[column].isin(missing_values).sum()}</b>"))

        else:

            display(
                HTML(f"> No null values ​​exist in column <i>'{column}'</i>"))

# Function to detect outlier boundaries with optional clamping of lower bound to zero
def outlier_limit_bounds(df, column, bound='both', clamp_zero=False):
    """
    Detects outlier thresholds based on the IQR method and returns rows beyond those limits.

    Parameters:
    - df (DataFrame): The input DataFrame.
    - column (str): The name of the numerical column to analyze.
    - bound (str): One of 'both', 'lower', or 'upper' to indicate which bounds to evaluate.
    - clamp_zero (bool): If True, clamps the lower bound to zero (useful for non-negative metrics).

    Returns:
    DataFrame(s): Rows identified as outliers, depending on the bound selected.
    """

    q1, q3 = df[column].quantile([0.25, 0.75])
    iqr = q3 - q1

    lower_bound = max(q1 - 1.5 * iqr, 0) if clamp_zero else q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    display(HTML(f"> Outlier thresholds for <i>'{column}'</i>: \n"
                 f"> Lower = <b>{lower_bound:.3f}</b>, > Upper = <b>{upper_bound:.3f}</b>"))

    if bound not in ['both', 'lower', 'upper']:
        display(HTML(f"> Invalid 'bound' parameter. Use <b>'both'</b>, <b>'upper'</b>, or <b>'lower'</b>."))
        return

    outliers = pd.DataFrame()
    
    if bound in ['both', 'lower']:
        lower_outliers = df[df[column] < lower_bound]
        if lower_outliers.empty:
            display(HTML(f"> <b>No</b> lower outliers found in column <i>'{column}'</i>."))
        outliers = pd.concat([outliers, lower_outliers])

    if bound in ['both', 'upper']:
        upper_outliers = df[df[column] > upper_bound]
        if upper_outliers.empty:
            display(HTML(f"> <b>No</b> upper outliers found in column <i>'{column}'</i>."))
        outliers = pd.concat([outliers, upper_outliers])

    display(HTML(f"- - -"))
    display(HTML(f"> Outliers:"))

    return outliers if not outliers.empty else None


# Function to plot a histogram of datetime values with full timestamp precision (YYYY-MM-DD HH:MM:SS),
# including vertical lines for mean and median.
def plotly_frequency_datetime_plotlypx(ds, bins=30, density=False, color='grey', title='Datetime Histogram', xlabel='Datetime', ylabel='Frequency', rotation='auto'):
    """
    Histogram for datetime values with X-axis showing precise timestamps (YYYY-MM-DD HH:MM:SS).
    - ds: Serie datetime
    - bins: número de bins o array de bordes
    - density: True = densidad, False = frecuencia
    """
    s = pd.Series(ds).dropna()

    # 1) Normalize to UTC and convert to integers: epoch to MILLISECONDS (int64)
    # -> this completely avoids ns/fractions
    dsu = pd.to_datetime(s, utc=True)
    ds_ms = (dsu.astype('int64') // 1_000_000).astype('int64')  # ns -> ms (enteros)

    # 2) Histogram over integers (ms)
    hist, bin_edges_ms = np.histogram(ds_ms, bins=bins, density=density)

    # 3) Centers and width in ms (INTEGER, no floats)
    bin_edges_ms = bin_edges_ms.astype('int64')
    bin_centers_ms = (bin_edges_ms[:-1] + bin_edges_ms[1:]) // 2  # integer midpoint
    widths_ms = (bin_edges_ms[1:] - bin_edges_ms[:-1]).astype('int64')

    #4) Convert ONLY here to datetime (ms) for X axis. No ns -> no warning.
    bin_centers_dt = pd.to_datetime(bin_centers_ms, unit='ms', utc=True)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=bin_centers_dt, y=hist.astype(float), marker_color=color, opacity=0.7, width=widths_ms,  # Plotly expects milliseconds as a number on datetime axes
                         name='Histogram'))

    # 5) Mean and median lines: calculate in ms (int) and convert to datetime (ms)
    mean_ms = int(ds_ms.mean())           # average in ms
    median_ms = int(np.median(ds_ms))     # median in ms
    mean_dt = pd.to_datetime(mean_ms, unit='ms', utc=True)
    median_dt = pd.to_datetime(median_ms, unit='ms', utc=True)

    y_top = float(max(hist) * 1.1) if len(hist) else 1.0
    fig.add_trace(go.Scatter(x=[mean_dt, mean_dt], y=[0, y_top], mode='lines', line=dict(color='red', dash='dash'), name='Mean'))
    fig.add_trace(go.Scatter(x=[median_dt, median_dt], y=[0, y_top], mode='lines', line=dict(color='blue', dash='dashdot'), name='Median'))

    # 6) Layout
    fig.update_layout(
        title=title,
        xaxis_title=xlabel,
        yaxis_title=ylabel if ylabel else ('Density' if density else 'Frequency'),
        template='plotly_white',
        bargap=0.05,
        width=1200, height=500
    )
    fig.update_xaxes(tickformat='%Y-%m-%d %H:%M:%S')  # clean format
    
    # 7) Automatic rotation (or fixed if you enter a number)
    if rotation == 'auto':
        # Simple heuristic by number of bins (robust and sufficient in practice)
        n = len(bin_centers_dt)
        if n <= 10:
            angle = 0
        elif n <= 20:
            angle = 30
        elif n <= 40:
            angle = 45
        elif n <= 80:
            angle = 60
        else:
            angle = 90
        fig.update_xaxes(tickangle=angle)
    elif isinstance(rotation, (int, float)):
        fig.update_xaxes(tickangle=float(rotation))
    # (if rotation is None or an unexpected value, we leave Plotly's default)
    
    fig.show()

# Function to plot a vertical bar chart using Plotly Express
def plot_vertical_bar_plotpx(df, x: str, y: str, hue: str | None = None,         # categorical column for grouping (like hue in seaborn)
                               title: str = '', xlabel: str = '', ylabel: str = '', sort: bool = True, height: int = 500, width: int = 1200, color: str = 'grey'):
    """
    Plots a horizontal bar chart with Plotly Express.

    Parameters:
    - df (DataFrame): Input DataFrame.
    - x (str): Column name for x-axis values (numeric).
    - y (str): Column name for y-axis categories.
    - hue (str, optional): Column for coloring/grouping (categorical).
    - title (str): Chart title.
    - xlabel (str): X-axis label.
    - ylabel (str): Y-axis label.
    - sort (bool): If True, sorts bars by x descending.
    - height (int): Figure height.
    - width (int): Figure width.
    - color (str): Default bar color if no hue is given.
    """

    data = df.copy()
    if sort:
        data = data.sort_values(by=x, ascending=True)

    if hue:
        fig = px.bar(
            data,
            x=x,
            y=y,
            color=hue,             # uses categoric columns
            title=title,
            height=height,
            width=width
        )
    else:
        fig = px.bar(
            data,
            x=x,
            y=y,
            title=title,
            height=height,
            width=width,
            color_discrete_sequence=[color]
        )

    fig.update_layout(
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        template='plotly_white'
    )

    fig.show()

# Function to plot a horizontal bar chart using Plotly Express
def plot_horizontal_bar_plotpx(df, x: str, y: str, hue: str | None = None,         # categorical column for grouping (like hue in seaborn)
                               title: str = '', xlabel: str = '', ylabel: str = '', sort: bool = True, height: int = 500, width: int = 1200, color: str = 'grey'):
    """
    Plots a horizontal bar chart with Plotly Express.

    Parameters:
    - df (DataFrame): Input DataFrame.
    - x (str): Column name for x-axis values (numeric).
    - y (str): Column name for y-axis categories.
    - hue (str, optional): Column for coloring/grouping (categorical).
    - title (str): Chart title.
    - xlabel (str): X-axis label.
    - ylabel (str): Y-axis label.
    - sort (bool): If True, sorts bars by x descending.
    - height (int): Figure height.
    - width (int): Figure width.
    """

    data = df.copy()
    if sort:
        data = data.sort_values(by=x, ascending=True)
        
    if hue:
        fig = px.bar(
            data,
            x=x,
            y=y,
            orientation='h',
            color=hue,             # uses categoric columns
            title=title,
            height=height,
            width=width
        )
    else:
        fig = px.bar(
            data,
            x=x,
            y=y,
            title=title,
            height=height,
            width=width,
            color_discrete_sequence=[color]
        )

    fig.update_layout(
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        template='plotly_white'
    )

    fig.show()


### 🔁 __3. Data Loading__

In [3]:
df_fs = pd.read_csv('../data/raw/logs_exp_us.csv', sep='\t', header='infer', keep_default_na=False)

### 🧹 __4. Data Cleanup__

##### **4.1** Data Overview

In [4]:
df_fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


##### **4.2** Standardization of column heading formats (Lower case, snake case)

In [5]:
df_fs.columns = df_fs.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_fs.columns

Index(['eventname', 'deviceidhash', 'eventtimestamp', 'expid'], dtype='object')

##### **4.3** Standardizing Dataframe String values.

In [6]:
df_fs['eventname'] = df_fs['eventname'].str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)
df_fs

,eventname,deviceidhash,eventtimestamp,expid
0,mainscreenappear,4575588528974610257,1564029816,246
1,mainscreenappear,7416695313311560658,1564053102,246
2,paymentscreensuccessful,3518123091307005509,1564054127,248
3,cartscreenappear,3518123091307005509,1564054127,248
4,paymentscreensuccessful,6217807653094995999,1564055322,248
...,...,...,...,...
244121,mainscreenappear,4599628364049201812,1565212345,247
244122,mainscreenappear,5849806612437486590,1565212439,246
244123,mainscreenappear,5746969938801999050,1565212483,246
244124,mainscreenappear,5746969938801999050,1565212498,246


##### **4.4** Explicit Duplicate Removal

In [7]:
# Show explicit duplicates amount
display(HTML(f"> Explicit duplicates amount Dataframe <i>'df_fs'</i>: <b>{df_fs.duplicated().sum()}</b>"))

In [8]:
# Delete explicit duplicated rows
df_fs = df_fs.drop_duplicates().reset_index(drop=True)

display(HTML(f"> Explicit duplicates amount Dataframe <i>'df_fs'</i>: <b>{df_fs.duplicated().sum()}</b>"))

##### **4.5** Missing Value Analysis

In [9]:
missing_values_check(df_fs)

##### **4.6** Casting Datatypes

In [10]:
# df_fs 'eventname' to category
df_fs['eventname'] = df_fs['eventname'].astype('category')
df_fs['eventname'].dtype


CategoricalDtype(categories=['cartscreenappear', 'mainscreenappear', 'offersscreenappear', 'paymentscreensuccessful', 'tutorial'], ordered=False, categories_dtype=object)

In [11]:
# df_fs 'eventtimestamp' to datetime
df_fs['eventtimestamp'] = pd.to_datetime(df_fs['eventtimestamp'], unit='s', errors='coerce')
df_fs = df_fs.rename(columns={'eventtimestamp': 'datetime'})
df_fs.dtypes

eventname             category
deviceidhash             int64
datetime        datetime64[ns]
expid                    int64
dtype: object

In [12]:
df_fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243713 entries, 0 to 243712
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   eventname     243713 non-null  category      
 1   deviceidhash  243713 non-null  int64         
 2   datetime      243713 non-null  datetime64[ns]
 3   expid         243713 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(2)
memory usage: 5.8 MB


##### **4.7** Feature Engineering

In [13]:
# Add date column
df_fs['date'] = df_fs['datetime'].dt.date

# Add time column
df_fs['time'] = df_fs['datetime'].dt.time

df_fs

,eventname,deviceidhash,datetime,expid,date,time
0,mainscreenappear,4575588528974610257,2019-07-25 04:43:36,246,2019-07-25,04:43:36
1,mainscreenappear,7416695313311560658,2019-07-25 11:11:42,246,2019-07-25,11:11:42
2,paymentscreensuccessful,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25,11:28:47
3,cartscreenappear,3518123091307005509,2019-07-25 11:28:47,248,2019-07-25,11:28:47
4,paymentscreensuccessful,6217807653094995999,2019-07-25 11:48:42,248,2019-07-25,11:48:42
...,...,...,...,...,...,...
243708,mainscreenappear,4599628364049201812,2019-08-07 21:12:25,247,2019-08-07,21:12:25
243709,mainscreenappear,5849806612437486590,2019-08-07 21:13:59,246,2019-08-07,21:13:59
243710,mainscreenappear,5746969938801999050,2019-08-07 21:14:43,246,2019-08-07,21:14:43
243711,mainscreenappear,5746969938801999050,2019-08-07 21:14:58,246,2019-08-07,21:14:58


### 🧮 5. EDA Exploratory Data Analysis

##### **5.1** Descriptive Analysis

In [14]:
df_fs.describe(include='all')

,eventname,deviceidhash,datetime,expid,date,time
count,243713,2.437130e+05,243713,243713.000000,243713,243713
unique,5,NaN,NaN,NaN,14,69824
top,mainscreenappear,NaN,NaN,NaN,2019-08-01,12:29:55
freq,119101,NaN,NaN,NaN,36141,19
mean,NaN,4.627963e+18,2019-08-04 10:19:17.987665920,247.022161,NaN,NaN
min,NaN,6.888747e+15,2019-07-25 04:43:36,246.000000,NaN,NaN
25%,NaN,2.372212e+18,2019-08-02 14:36:45,246.000000,NaN,NaN
50%,NaN,4.623192e+18,2019-08-04 11:51:00,247.000000,NaN,NaN
75%,NaN,6.932517e+18,2019-08-06 06:56:24,248.000000,NaN,NaN
max,NaN,9.222603e+18,2019-08-07 21:15:17,248.000000,NaN,NaN


##### **5.2** Data Visualization: Data Analysis.

5.2.1 How many events are in the logs?

In [15]:
# Show total number of events within the records
display(HTML(f"> <b>Total</b> number of <b>events</b> within the records: <i>{df_fs.shape[0]}</i> (events)"))

5.2.2 How many users are there in the records?

In [16]:
# Show total number users within the records
display(HTML(f"> <b>Total</b> number of <b>users</b> within the records: <i>{df_fs['deviceidhash'].nunique()}</i> (users)"))

5.2.3 What is the average number of events per user?

In [17]:
# Show the average number of events per user
display(HTML(f"> <b>Average</b> number of <b>events</b> per user: <i>{df_fs.groupby('deviceidhash')['eventname'].count().mean():.3f}</i> (events)"))

5.2.4 What time period does the data cover? Find the maximum and minimum dates. Plot a histogram by date and time. What period do the data actually represent?

In [18]:
# Show the time period
display(HTML(f"> <b>Earliest date</b> within the records: <i>{df_fs['date'].min()}</i>"))
display(HTML(f"> <b>Latest date</b> within the records: <i>{df_fs['date'].max()}</i>"))

In [19]:
# Plot a histogram by date and time.
plotly_frequency_datetime_plotlypx(df_fs['datetime'], bins=500, density=False, color='grey', title='Event Distribution with Full Timestamp Precision', xlabel='Date-Time', ylabel='Frequency')

In [20]:
# Find the point at which the data begins to be complete and ignore the previous section. What period does the data actually represent?
df_fs["datetime"] = df_fs["datetime"].astype("int64") // 10**9
df_fs


,eventname,deviceidhash,datetime,expid,date,time
0,mainscreenappear,4575588528974610257,1564029816,246,2019-07-25,04:43:36
1,mainscreenappear,7416695313311560658,1564053102,246,2019-07-25,11:11:42
2,paymentscreensuccessful,3518123091307005509,1564054127,248,2019-07-25,11:28:47
3,cartscreenappear,3518123091307005509,1564054127,248,2019-07-25,11:28:47
4,paymentscreensuccessful,6217807653094995999,1564055322,248,2019-07-25,11:48:42
...,...,...,...,...,...,...
243708,mainscreenappear,4599628364049201812,1565212345,247,2019-08-07,21:12:25
243709,mainscreenappear,5849806612437486590,1565212439,246,2019-08-07,21:13:59
243710,mainscreenappear,5746969938801999050,1565212483,246,2019-08-07,21:14:43
243711,mainscreenappear,5746969938801999050,1565212498,246,2019-08-07,21:14:58


In [21]:
# Analyzing outliers
outlier_limit_bounds(df_fs, column='datetime', bound='both', clamp_zero=False)

,eventname,deviceidhash,datetime,expid,date,time
0,mainscreenappear,4575588528974610257,1564029816,246,2019-07-25,04:43:36
1,mainscreenappear,7416695313311560658,1564053102,246,2019-07-25,11:11:42
2,paymentscreensuccessful,3518123091307005509,1564054127,248,2019-07-25,11:28:47
3,cartscreenappear,3518123091307005509,1564054127,248,2019-07-25,11:28:47
4,paymentscreensuccessful,6217807653094995999,1564055322,248,2019-07-25,11:48:42
...,...,...,...,...,...,...
90,mainscreenappear,2793988848638831992,1564252142,247,2019-07-27,18:29:02
91,offersscreenappear,4284716907364183621,1564253762,247,2019-07-27,18:56:02
92,mainscreenappear,5218365729556903805,1564255514,247,2019-07-27,19:25:14
93,mainscreenappear,5218365729556903805,1564255522,247,2019-07-27,19:25:22


In [22]:
display(HTML(f"> Lower = {pd.to_datetime(1564279636.500, unit="s", utc=True)}"))
display(HTML(f"> Lower = {pd.to_datetime(1565551552.500, unit="s", utc=True)}"))

In [23]:
df_fs = df_fs.copy()
df_fs = df_fs.loc[(df_fs['datetime'] > 1564297636.500) & (df_fs['datetime'] < 1565569552.500), :]
df_fs['datetime'] = pd.to_datetime(df_fs['datetime'], unit='s', errors='coerce')
df_fs

,eventname,deviceidhash,datetime,expid,date,time
110,offersscreenappear,5287660122200561379,2019-07-28 07:24:12,247,2019-07-28,07:24:12
111,mainscreenappear,4601930136642303959,2019-07-28 07:25:42,246,2019-07-28,07:25:42
112,mainscreenappear,4078637421796153763,2019-07-28 07:31:58,248,2019-07-28,07:31:58
113,mainscreenappear,2851587046453391812,2019-07-28 07:40:40,246,2019-07-28,07:40:40
114,mainscreenappear,9111985047183779356,2019-07-28 07:48:19,247,2019-07-28,07:48:19
...,...,...,...,...,...,...
243708,mainscreenappear,4599628364049201812,2019-08-07 21:12:25,247,2019-08-07,21:12:25
243709,mainscreenappear,5849806612437486590,2019-08-07 21:13:59,246,2019-08-07,21:13:59
243710,mainscreenappear,5746969938801999050,2019-08-07 21:14:43,246,2019-08-07,21:14:43
243711,mainscreenappear,5746969938801999050,2019-08-07 21:14:58,246,2019-08-07,21:14:58


In [24]:
# Plot a histogram by date and time, no outliers.
plotly_frequency_datetime_plotlypx(df_fs['datetime'], bins=500, density=False, color='grey', title='Event Distribution with Full Timestamp Precision', xlabel='Date-Time', ylabel='Frequency')

In [25]:
# Number and proportion of records lost after excluding older records
records = df_fs.shape[0]
cutoff = pd.to_datetime('2019-08-01 00:00:00')  # Make cutoff timezone-naive
# Check if any records are before the cutoff (should be zero after filtering)
excluded = df_fs.loc[df_fs['datetime'] < cutoff, :].shape[0]
display(HTML(f"> Total number of records: {records}"))
display(HTML(f"> Number of records to be excluded: {excluded}"))
display(HTML(f"> Proportion of records to be excluded: {(excluded * 100) / records:.3f} %"))

In [26]:
# Data prior to August 1, 2019, have incomplete records and are not included in the analysis to avoid bias.
df_fs = df_fs.loc[df_fs['datetime'] > cutoff, :]
df_fs

,eventname,deviceidhash,datetime,expid,date,time
2826,tutorial,3737462046622621720,2019-08-01 00:07:28,246,2019-08-01,00:07:28
2827,mainscreenappear,3737462046622621720,2019-08-01 00:08:00,246,2019-08-01,00:08:00
2828,mainscreenappear,3737462046622621720,2019-08-01 00:08:55,246,2019-08-01,00:08:55
2829,offersscreenappear,3737462046622621720,2019-08-01 00:08:58,246,2019-08-01,00:08:58
2830,mainscreenappear,1433840883824088890,2019-08-01 00:08:59,247,2019-08-01,00:08:59
...,...,...,...,...,...,...
243708,mainscreenappear,4599628364049201812,2019-08-07 21:12:25,247,2019-08-07,21:12:25
243709,mainscreenappear,5849806612437486590,2019-08-07 21:13:59,246,2019-08-07,21:13:59
243710,mainscreenappear,5746969938801999050,2019-08-07 21:14:43,246,2019-08-07,21:14:43
243711,mainscreenappear,5746969938801999050,2019-08-07 21:14:58,246,2019-08-07,21:14:58


In [27]:
# Plot a histogram by date and time, just complete records.
plotly_frequency_datetime_plotlypx(df_fs['datetime'], bins=500, density=False, color='grey', title='Event Distribution with Full Timestamp Precision', xlabel='Date-Time', ylabel='Frequency')

### 📝 6. Events Funnel Analysis.

##### **6.1** Review the events listed in the logs and their frequency of occurrence. Sort them by frequency.

In [28]:
# Events frequency and sorting
df_fs_events = df_fs.groupby(['date', df_fs['datetime'].dt.hour]).size()
df_fs_events.name = 'events'
df_fs_events = df_fs_events.reset_index()
df_fs_events['datetime'] = df_fs_events['date'].astype(str) + '_' + df_fs_events['datetime'].astype(str)
df_fs_events = df_fs_events.drop(columns='date')
df_fs_events

,datetime,events
0,2019-08-01_0,192
1,2019-08-01_1,228
2,2019-08-01_2,328
3,2019-08-01_3,657
4,2019-08-01_4,837
...,...,...
161,2019-08-07_17,1906
162,2019-08-07_18,1679
163,2019-08-07_19,1507
164,2019-08-07_20,949


In [29]:
plot_vertical_bar_plotpx(df_fs_events, x='datetime', y='events', title='Events Frequency', xlabel='Date', ylabel='Events', sort=True)

##### **6.2** Find the number of users who performed each of these actions. Sort the events by the number of users. Calculate the percentage of users who performed the action at least once.

In [30]:
# Show users per actions
df_fs_events = df_fs.groupby('eventname', observed=True).agg(events=('eventname', 'count'), users=('deviceidhash', 'nunique')).sort_values(by='users', ascending=False).reset_index()
df_fs_events

,eventname,events,users
0,mainscreenappear,117328,7419
1,offersscreenappear,46333,4593
2,cartscreenappear,42303,3734
3,paymentscreensuccessful,33918,3539
4,tutorial,1005,840


In [31]:
plot_horizontal_bar_plotpx(df_fs_events, x='users', y='eventname', title='Conversion Funnel', xlabel='Users', ylabel='Events', sort=True)

In [32]:
# Show percentage of users that perfomed the action at least once
df_fs_events['conversion_event'] = ((df_fs_events['users'] / df_fs_events['events']) * 100).round(3)
df_fs_events

,eventname,events,users,conversion_event
0,mainscreenappear,117328,7419,6.323
1,offersscreenappear,46333,4593,9.913
2,cartscreenappear,42303,3734,8.827
3,paymentscreensuccessful,33918,3539,10.434
4,tutorial,1005,840,83.582


`LSPL`
- Order in which the events occurred: mainscreenappear, offersscreenappear, cartscreenappear, paymentscreensuccessful, tutorial.
- tutorial event seems not to be part of the same sequence, proper sequence should be clarified ans confirmed by the customer(mainscreenappear, offersscreenappear, cartscreenappear, paymentscreensuccessful)

##### **6.3**  Use the event funnel to find the percentage of users who move from one stage to the next.

In [33]:
# Show Total convesion rate
df_fs_events['totalconversionrate'] = ((df_fs_events['users'] / df_fs_events.loc[0, 'users']) * 100).round(3)
df_fs_events

,eventname,events,users,conversion_event,totalconversionrate
0,mainscreenappear,117328,7419,6.323,100.000
1,offersscreenappear,46333,4593,9.913,61.909
2,cartscreenappear,42303,3734,8.827,50.330
3,paymentscreensuccessful,33918,3539,10.434,47.702
4,tutorial,1005,840,83.582,11.322


In [34]:
# Show Total convesion next stage rate
df_fs_events['conversionstagerate'] = ((df_fs_events['users'] / df_fs_events['users'].shift(1)) * 100).round(3)
df_fs_events

,eventname,events,users,conversion_event,totalconversionrate,conversionstagerate
0,mainscreenappear,117328,7419,6.323,100.000,NaN
1,offersscreenappear,46333,4593,9.913,61.909,61.909
2,cartscreenappear,42303,3734,8.827,50.330,81.298
3,paymentscreensuccessful,33918,3539,10.434,47.702,94.778
4,tutorial,1005,840,83.582,11.322,23.736


In [35]:
df_fs_events['droprate'] = (100 - df_fs_events['conversionstagerate']).round(3)
df_fs_events

,eventname,events,users,conversion_event,totalconversionrate,conversionstagerate,droprate
0,mainscreenappear,117328,7419,6.323,100.000,NaN,NaN
1,offersscreenappear,46333,4593,9.913,61.909,61.909,38.091
2,cartscreenappear,42303,3734,8.827,50.330,81.298,18.702
3,paymentscreensuccessful,33918,3539,10.434,47.702,94.778,5.222
4,tutorial,1005,840,83.582,11.322,23.736,76.264


In [36]:
plot_horizontal_bar_plotpx(df_fs_events, x='totalconversionrate', y='eventname', title='Conversion Rate', xlabel='Conversion Rate', ylabel='Events', sort=True)

In [37]:
plot_horizontal_bar_plotpx(df_fs_events, x='conversionstagerate', y='eventname', title='Conversion Rate per Stage', xlabel='Conversion Rate', ylabel='Events', sort=True)

In [38]:
plot_horizontal_bar_plotpx(df_fs_events, x='droprate', y='eventname', title='Drop Rate per Stage', xlabel='Drop Rate', ylabel='Events', sort=True)

In [39]:
# Percentage of users who complete the entire journey from their first interaction to making a purchase
display(HTML(f"> User rate through complete funnel: {((df_fs_events.loc[3, 'users'] / df_fs_events.loc[0, 'users']) * 100):.3f}%"))

### 📈 7. __Statistical Data Analysis__.

### 7.1  Inferential Tests.

#### 7.1.0  Data Analysis prior to A/B test

In [40]:
df_fs

,eventname,deviceidhash,datetime,expid,date,time
2826,tutorial,3737462046622621720,2019-08-01 00:07:28,246,2019-08-01,00:07:28
2827,mainscreenappear,3737462046622621720,2019-08-01 00:08:00,246,2019-08-01,00:08:00
2828,mainscreenappear,3737462046622621720,2019-08-01 00:08:55,246,2019-08-01,00:08:55
2829,offersscreenappear,3737462046622621720,2019-08-01 00:08:58,246,2019-08-01,00:08:58
2830,mainscreenappear,1433840883824088890,2019-08-01 00:08:59,247,2019-08-01,00:08:59
...,...,...,...,...,...,...
243708,mainscreenappear,4599628364049201812,2019-08-07 21:12:25,247,2019-08-07,21:12:25
243709,mainscreenappear,5849806612437486590,2019-08-07 21:13:59,246,2019-08-07,21:13:59
243710,mainscreenappear,5746969938801999050,2019-08-07 21:14:43,246,2019-08-07,21:14:43
243711,mainscreenappear,5746969938801999050,2019-08-07 21:14:58,246,2019-08-07,21:14:58


In [41]:
# Amount of users are in each group
df_fs_users_group = df_fs.groupby('expid')['deviceidhash'].nunique().reset_index()
df_fs_users_group = df_fs_users_group.rename(columns={'expid': 'group', 'deviceidhash': 'users'})
df_fs_users_group

,group,users
0,246,2484
1,247,2513
2,248,2537


#### 7.1.1  A/A Control groups Analysis

Hypothesis(0): Control group A1 (246) and control group A2 (247) there is no statistical significant difference.   
Hypothesis(1): Control group A1 (246) and control group A2 (247) there is statistical significant difference.

In [42]:
# Select the most popular event. For each of the control groups, determine the number of users who performed this action. Calculate the percentage.
events = df_fs['eventname'].value_counts()
events.name = 'events'
events = events.reset_index()
events

,eventname,events
0,mainscreenappear,117328
1,offersscreenappear,46333
2,cartscreenappear,42303
3,paymentscreensuccessful,33918
4,tutorial,1005


In [43]:
popular_event = df_fs['eventname'].mode()[0]
popular_event

'mainscreenappear'

In [44]:
total_users_group = df_fs.groupby('expid')['deviceidhash'].nunique().reset_index()
total_users_group.columns = ['group', 'total_users']
total_users_group

,group,total_users
0,246,2484
1,247,2513
2,248,2537


In [46]:
event_users_group = df_fs.groupby(['expid', 'eventname'], observed=False)['deviceidhash'].nunique().reset_index()
event_users_group.columns = ['group', 'eventname', 'event_users']
event_users_group

,group,eventname,event_users
0,246,cartscreenappear,1266
1,246,mainscreenappear,2450
2,246,offersscreenappear,1542
3,246,paymentscreensuccessful,1200
4,246,tutorial,278
5,247,cartscreenappear,1238
6,247,mainscreenappear,2476
7,247,offersscreenappear,1520
8,247,paymentscreensuccessful,1158
9,247,tutorial,283


In [47]:
df_event_group = event_users_group.merge(total_users_group, on='group')
df_event_group['eventrate'] = (df_event_group['event_users'] / df_event_group['total_users'] * 100).round(3)
df_event_group

,group,eventname,event_users,total_users,eventrate
0,246,cartscreenappear,1266,2484,50.966
1,246,mainscreenappear,2450,2484,98.631
2,246,offersscreenappear,1542,2484,62.077
3,246,paymentscreensuccessful,1200,2484,48.309
4,246,tutorial,278,2484,11.192
5,247,cartscreenappear,1238,2513,49.264
6,247,mainscreenappear,2476,2513,98.528
7,247,offersscreenappear,1520,2513,60.485
8,247,paymentscreensuccessful,1158,2513,46.080
9,247,tutorial,283,2513,11.261


In [48]:
# α = 0.5
def control_groups_proportions_statistical_difference(df):
    
    events = df['eventname'].unique()
    
    for event in events:
        
        actions_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 246), 'event_users'],
                                  df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 247), 'event_users']])
        
        events_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 246), 'total_users'],
                                 df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 247), 'total_users']])
        
        display(HTML(f"> Event: <b>{event.upper()}</b>"))
        
        stats, p_value = proportions_ztest(actions_event, events_event, alternative="two-sided")
        display(HTML(f"> Z-statistic: {stats}"))
        display(HTML(f"> p-value: {p_value}"))
        
        if p_value <= 0.05:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that <b>conversion rate</b> between Group A1 (246) and Group A2 (247) are <b>different</b>."))
        else:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that <b>conversion rate</b> between Group A1 (246) and Group A2 (247) are different."))
        
        print()
 

In [49]:
control_groups_proportions_statistical_difference(df_event_group)

In [50]:
# Confirm that the groups were divided correctly
pivot_rates = df_event_group.pivot(index='eventname', columns='group', values='eventrate')
pivot_rates

group,246,247,248
eventname,,,
cartscreenappear,50.966,49.264,48.482
mainscreenappear,98.631,98.528,98.266
offersscreenappear,62.077,60.485,60.347
paymentscreensuccessful,48.309,46.080,46.551
tutorial,11.192,11.261,10.997


#### 7.1.2  A1/B Control-Test groups Analysis

Hypothesis(0): Control group A1 (246) and test group B (248) there is no statistical significant difference.   
Hypothesis(1): Control group A1 (246) and test group B (248) there is statistical significant difference.

In [51]:
def control_A1_test_proportions_statistical_difference(df):
    
    events = df['eventname'].unique()
    
    for event in events:
        
        actions_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 246), 'event_users'],
                                  df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 248), 'event_users']])
        
        events_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 246), 'total_users'],
                                 df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 248), 'total_users']])
        
        display(HTML(f"> Event: <b>{event.upper()}</b>"))
        
        stats, p_value = proportions_ztest(actions_event, events_event, alternative="two-sided")
        display(HTML(f"> Z-statistic: {stats}"))
        display(HTML(f"> p-value: {p_value}"))
        
        if p_value <= 0.05:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that <b>conversion rate</b> between Group A1 (246) and Group B (248) are <b>different</b>."))
        else:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that <b>conversion rate</b> between Group A1 (246) and Group B (248) are different."))
        
        print()

In [52]:
control_A1_test_proportions_statistical_difference(df_event_group)

#### 7.1.3  A2/B Control-Test groups Analysis

Hypothesis(0): Control group A1 (247) and test group B (248) there is no statistical significant difference.   
Hypothesis(1): Control group A1 (247) and test group B (248) there is statistical significant difference.

In [53]:
def control_A2_test_proportions_statistical_difference(df):
    
    events = df['eventname'].unique()
    
    for event in events:
        
        actions_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 247), 'event_users'],
                                  df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 248), 'event_users']])
        
        events_event = np.array([df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 247), 'total_users'],
                                 df_event_group.loc[(df_event_group['eventname'] == event) & (df_event_group['group'] == 248), 'total_users']])
        
        display(HTML(f"> Event: <b>{event.upper()}</b>"))
        
        stats, p_value = proportions_ztest(actions_event, events_event, alternative="two-sided")
        display(HTML(f"> Z-statistic: {stats}"))
        display(HTML(f"> p-value: {p_value}"))
        
        if p_value <= 0.05:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that <b>conversion rate</b> between Group A2 (247) and Group B (248) are <b>different</b>."))
        else:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that <b>conversion rate</b> between Group A2 (247) and Group B (248) are different."))
        
        print()

In [54]:
control_A2_test_proportions_statistical_difference(df_event_group)

#### 7.1.4  AU/B Control-Test groups Analysis

Hypothesis(0): Control group AU (246/247) and test group B (248) there is no statistical significant difference.   
Hypothesis(1): Control group AU (246/247) and test group B (248) there is statistical significant difference.

In [55]:
df_event_group

,group,eventname,event_users,total_users,eventrate
0,246,cartscreenappear,1266,2484,50.966
1,246,mainscreenappear,2450,2484,98.631
2,246,offersscreenappear,1542,2484,62.077
3,246,paymentscreensuccessful,1200,2484,48.309
4,246,tutorial,278,2484,11.192
5,247,cartscreenappear,1238,2513,49.264
6,247,mainscreenappear,2476,2513,98.528
7,247,offersscreenappear,1520,2513,60.485
8,247,paymentscreensuccessful,1158,2513,46.080
9,247,tutorial,283,2513,11.261


In [56]:
df_event_group_AU = df_event_group.loc[(df_event_group['group'] == 246) | (df_event_group['group'] == 247), :]
df_event_group_AU

,group,eventname,event_users,total_users,eventrate
0,246,cartscreenappear,1266,2484,50.966
1,246,mainscreenappear,2450,2484,98.631
2,246,offersscreenappear,1542,2484,62.077
3,246,paymentscreensuccessful,1200,2484,48.309
4,246,tutorial,278,2484,11.192
5,247,cartscreenappear,1238,2513,49.264
6,247,mainscreenappear,2476,2513,98.528
7,247,offersscreenappear,1520,2513,60.485
8,247,paymentscreensuccessful,1158,2513,46.080
9,247,tutorial,283,2513,11.261


In [57]:
tu = df_event_group_AU['total_users'].unique().sum()
tu

np.int64(4997)

In [59]:
df_event_group_AU = df_event_group_AU.groupby('eventname', observed=False)['event_users'].sum().reset_index()
df_event_group_AU['group'] = '246_247'
df_event_group_AU['total_users'] = tu
df_event_group_AU['eventrate'] = ((df_event_group_AU['event_users'] / df_event_group_AU['total_users']) * 100).round(3)
df_event_group_AU

,eventname,event_users,group,total_users,eventrate
0,cartscreenappear,2504,246_247,4997,50.110
1,mainscreenappear,4926,246_247,4997,98.579
2,offersscreenappear,3062,246_247,4997,61.277
3,paymentscreensuccessful,2358,246_247,4997,47.188
4,tutorial,561,246_247,4997,11.227


In [60]:
df_event_group_B = df_event_group.loc[(df_event_group['group'] == 248), :]
df_event_group_B = df_event_group_B.copy()
df_event_group_B['group'] = df_event_group_B['group'].astype(str)
df_event_group_B

,group,eventname,event_users,total_users,eventrate
10,248,cartscreenappear,1230,2537,48.482
11,248,mainscreenappear,2493,2537,98.266
12,248,offersscreenappear,1531,2537,60.347
13,248,paymentscreensuccessful,1181,2537,46.551
14,248,tutorial,279,2537,10.997


In [61]:
df_event_group_AB = pd.concat([df_event_group_AU, df_event_group_B], axis=0, ignore_index=True)
df_event_group_AB

,eventname,event_users,group,total_users,eventrate
0,cartscreenappear,2504,246_247,4997,50.110
1,mainscreenappear,4926,246_247,4997,98.579
2,offersscreenappear,3062,246_247,4997,61.277
3,paymentscreensuccessful,2358,246_247,4997,47.188
4,tutorial,561,246_247,4997,11.227
5,cartscreenappear,1230,248,2537,48.482
6,mainscreenappear,2493,248,2537,98.266
7,offersscreenappear,1531,248,2537,60.347
8,paymentscreensuccessful,1181,248,2537,46.551
9,tutorial,279,248,2537,10.997


In [62]:
def control_test_proportions_statistical_difference(df):
    
    events = df['eventname'].unique()
    
    for event in events:
        
        actions_event = np.array([df_event_group_AB.loc[(df_event_group_AB['eventname'] == event) & (df_event_group_AB['group'] == '246_247'), 'event_users'],
                                  df_event_group_AB.loc[(df_event_group_AB['eventname'] == event) & (df_event_group_AB['group'] == '248'), 'event_users']])
        
        events_event = np.array([df_event_group_AB.loc[(df_event_group_AB['eventname'] == event) & (df_event_group_AB['group'] == '246_247'), 'total_users'],
                                 df_event_group_AB.loc[(df_event_group_AB['eventname'] == event) & (df_event_group_AB['group'] == '248'), 'total_users']])
        
        display(HTML(f"> Event: <b>{event.upper()}</b>"))
        
        stats, p_value = proportions_ztest(actions_event, events_event, alternative="two-sided")
        display(HTML(f"> Z-statistic: {stats}"))
        display(HTML(f"> p-value: {p_value}"))
        
        if p_value <= 0.05:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>rejected</b>, meaning there is enough statistical evidence that <b>conversion rate</b> between Group A (246_247) and Group B (248) are <b>different</b>."))
        else:
            display(HTML(f"> Null Hypothesis (<i>H₀</i>) is <b>not rejected</b>, meaning there is not enough statistical evidence that <b>conversion rate</b> between Group A (246_247) and Group B (248) are different."))
        
        print()

In [63]:
control_test_proportions_statistical_difference(df_event_group_AB)

#### 7.2 Results

- Significance level that have been set to test the statistical hypotheses mentioned above : α = 0.05
- Amount of statistical hypothesis tests that have been performed: 4 (A/A, A1/B, A2/B, A/B)
- What should the significance level be? Please specify if you want to change it.
  If α = 0.01, the results would be the same due to the p-value gotten from all the statistical tests (greater than α). In order to be changed
  the α value must be greater than p-value, acoording to the eventrate values no much differences are among froups, therefore p-value is allways
  greater than α.